## Creating a Regression Model

In this exercise, you will implement a regression model that uses features of a flight to predict how late or early it will arrive.

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

### Load Source Data
The data for this exercise is provided as a CSV file containing details of flights. The data includes specific characteristics (or *features*) for each flight, as well as a *label* column indicating how many minutes late or early the flight arrived.

You will load this data into a DataFrame and display it.

In [4]:
flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
  StructField("Late", IntegerType(), False),
])

data = spark.read.csv('wasb://spark@dbshepstor.blob.core.windows.net/data/flights.csv', schema=flightSchema, header=True)
data.show()

+----------+---------+-------+---------------+-------------+--------+--------+----+
DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|Late|
+----------+---------+-------+---------------+-------------+--------+--------+----+
 21| 2| WN| 10721| 13342| 26| 57| 1|
 13| 1| AA| 15016| 12892| 51| 27| 1|
 5| 5| FL| 10397| 11433| 9| 4| 0|
 22| 1| US| 11278| 14100| 35| 71| 1|
 23| 4| WN| 12451| 10693| 9| 5| 0|
 5| 7| AA| 11298| 15016| 39| 42| 1|
 4| 6| UA| 13930| 14307| 71| 58| 1|
 10| 3| 9E| 14307| 11433| 68| 140| 1|
 29| 7| UA| 14057| 14771| 130| 125| 1|
 14| 7| UA| 14771| 11292| 20| 42| 1|
 14| 6| UA| 12892| 11618| 6| -21| 0|
 16| 3| AA| 11298| 13204| 5| -7| 0|
 7| 3| VX| 14747| 14771| 143| 129| 1|
 20| 2| DL| 13204| 12478| 9| 5| 0|
 28| 5| DL| 11292| 13487| 1| -3| 0|
 17| 6| MQ| 11042| 13303| 12| 26| 1|
 3| 3| EV| 12266| 15016| 20| 3| 0|
 2| 4| DL| 10397| 13198| 32| 23| 0|
 4| 5| WN| 12889| 13342| 90| 80| 1|
 25| 6| UA| 10721| 12266| -5| -27| 0|
+----------+---------+-------+---------------+-------------+--------+--------+----+
only showing top 20 rows

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [6]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 445476 Testing Rows: 191607

### Prepare the Training Data
To train the regression model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **StringIndexer** class to generate a numeric category for each discrete **Carrier** string value, and then use the **VectorAssembler** class to transform the numeric features that would be available for a flight that hasn't yet arrived into a vector, and then rename the **ArrDelay** column to **label** as this is what we're going to try to predict.

*Note: This is a deliberately simple example. In reality you'd likely perform mulitple data preparation steps, and later in this course we'll examine how to encapsulate these steps in to a pipeline. For now, we'll just use the numeric features as they are to dewfine the traaining dataset.*

In [8]:
# Carrier is a string, and we need our features to be numeric - so we'll generate a numeric index for each distinct carrier string, and transform the dataframe to add that as a column
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
numTrain = carrierIndexer.fit(train).transform(train)

# Now we'll assemble a vector of all the numeric feature columns (other than ArrDelay, which we wouldn't have for enroute flights)
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "CarrierIdx", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
training = assembler.transform(numTrain).select(col("features"), (col("ArrDelay").cast("Int").alias("label")))
training.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
[1.0,1.0,10.0,103...| 113|
[1.0,1.0,10.0,103...| 2|
[1.0,1.0,10.0,105...| 41|
[1.0,1.0,10.0,107...| -2|
[1.0,1.0,10.0,107...| 30|
[1.0,1.0,10.0,110...| 7|
[1.0,1.0,10.0,111...| 86|
[1.0,1.0,10.0,111...| 27|
[1.0,1.0,10.0,111...| 56|
[1.0,1.0,10.0,111...| 55|
[1.0,1.0,10.0,111...| -12|
[1.0,1.0,10.0,111...| -4|
[1.0,1.0,10.0,111...| 44|
[1.0,1.0,10.0,111...| -19|
[1.0,1.0,10.0,111...| 66|
[1.0,1.0,10.0,112...| 50|
[1.0,1.0,10.0,112...| -7|
[1.0,1.0,10.0,114...| 26|
[1.0,1.0,10.0,114...| 37|
[1.0,1.0,10.0,114...| -11|
+--------------------+-----+
only showing top 20 rows

### Train a Regression Model
Next, you need to train a regression model using the training data. To do this, create an instance of the regression algorithm you want to use and use its **fit** method to train a model based on the training DataFrame. In this exercise, you will use a *Linear Regression* algorithm - though you can use the same technique for any of the regression algorithms supported in the spark.ml API.

In [10]:
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)
model = lr.fit(training)
print ("Model trained!")

Model trained!

### Prepare the Testing Data
Now that you have a trained model, you can test it using the testing data you reserved previously. First, you need to prepare the testing data in the same way as you did the training data by transforming the feature columns into a vector. This time you'll rename the **ArrDelay** column to **trueLabel**.

In [12]:
# Transform the test data to add the numeric carrier index
numTest = carrierIndexer.fit(test).transform(test)

# Generate the features vector and label
testing = assembler.transform(numTest).select(col("features"), (col("ArrDelay")).cast("Int").alias("trueLabel"))
testing.show()

+--------------------+---------+
 features|trueLabel|
+--------------------+---------+
[1.0,1.0,10.0,107...| 30|
[1.0,1.0,10.0,107...| -10|
[1.0,1.0,10.0,110...| 82|
[1.0,1.0,10.0,110...| -12|
[1.0,1.0,10.0,111...| 62|
[1.0,1.0,10.0,111...| 21|
[1.0,1.0,10.0,111...| -10|
[1.0,1.0,10.0,111...| 54|
[1.0,1.0,10.0,114...| -10|
[1.0,1.0,10.0,114...| -15|
[1.0,1.0,10.0,114...| 94|
[1.0,1.0,10.0,123...| -14|
[1.0,1.0,10.0,124...| -8|
[1.0,1.0,10.0,131...| -22|
[1.0,1.0,10.0,134...| 91|
[1.0,1.0,10.0,134...| -10|
[1.0,1.0,10.0,138...| 62|
[1.0,1.0,10.0,144...| 128|
[1.0,1.0,10.0,150...| 28|
[1.0,1.0,2.0,1039...| 28|
+--------------------+---------+
only showing top 20 rows

### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict arrival delay for flights where the label is unknown; but in this case you are using the test data which includes a known true label value, so you can compare the predicted number of minutes late or early to the actual arrival delay.

In [14]:
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show()

+--------------------+--------------------+---------+
 features| prediction|trueLabel|
+--------------------+--------------------+---------+
[1.0,1.0,10.0,107...| 39.60286773515003| 30|
[1.0,1.0,10.0,107...|-0.41544978784251896| -10|
[1.0,1.0,10.0,110...| 95.01445028768285| 82|
[1.0,1.0,10.0,110...| 3.40491333828737| -12|
[1.0,1.0,10.0,111...| 2.397078929461242| 62|
[1.0,1.0,10.0,111...| 8.200619465378784| 21|
[1.0,1.0,10.0,111...| 1.5607786846130676| -10|
[1.0,1.0,10.0,111...| 60.55597862754426| 54|
[1.0,1.0,10.0,114...| 2.7010684122714093| -10|
[1.0,1.0,10.0,114...| -0.8410190324504327| -15|
[1.0,1.0,10.0,114...| 52.75114844857194| 94|
[1.0,1.0,10.0,123...| -7.56976983280002| -14|
[1.0,1.0,10.0,124...| 2.1118031512914106| -8|
[1.0,1.0,10.0,131...| -0.840941177702339| -22|
[1.0,1.0,10.0,134...| 89.61641568528007| 91|
[1.0,1.0,10.0,134...| -1.7350189114176047| -10|
[1.0,1.0,10.0,138...| 76.14108250318239| 62|
[1.0,1.0,10.0,144...| 108.51177798303141| 128|
[1.0,1.0,10.0,150...| 24.320560617545528| 28|
[1.0,1.0,2.0,1039...| 29.21690831102692| 28|
+--------------------+--------------------+---------+
only showing top 20 rows

Looking at the result, the **prediction** column contains the predicted value for the label, and the **trueLabel** column contains the actual known value from the testing data. It looks like there is some variance between the predictions and the actual values (the individual differences are referred to as *residuals*)- later in this course you'll learn how to measure the accuracy of a model.